In [1]:
import os
import shutil

def reorganize_dataset(source_root, dest_root):

    if not os.path.exists(dest_root):
        os.makedirs(dest_root)

    for folder_name in os.listdir(source_root):
        folder_path = os.path.join(source_root, folder_name)
        
        if os.path.isdir(folder_path):
            for subfolder_name in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder_name)
                
                if os.path.isdir(subfolder_path):
                    dest_subfolder_path = os.path.join(dest_root, subfolder_name)
                    if not os.path.exists(dest_subfolder_path):
                        os.makedirs(dest_subfolder_path)
                    
                    for file_name in os.listdir(subfolder_path):
                        source_file = os.path.join(subfolder_path, file_name)
                        dest_file = os.path.join(dest_subfolder_path, file_name)
                        shutil.move(source_file, dest_file)
                    
                    print(f"Moved files from {subfolder_path} to {dest_subfolder_path}")

In [2]:
source_root = 'dataset/'
dest_root = 'reorganized_dataset/'
reorganize_dataset(source_root, dest_root)

Moved files from dataset/00\01_palm to reorganized_dataset/01_palm
Moved files from dataset/00\02_l to reorganized_dataset/02_l
Moved files from dataset/00\03_fist to reorganized_dataset/03_fist
Moved files from dataset/00\04_fist_moved to reorganized_dataset/04_fist_moved
Moved files from dataset/00\05_thumb to reorganized_dataset/05_thumb
Moved files from dataset/00\06_index to reorganized_dataset/06_index
Moved files from dataset/00\07_ok to reorganized_dataset/07_ok
Moved files from dataset/00\08_palm_moved to reorganized_dataset/08_palm_moved
Moved files from dataset/00\09_c to reorganized_dataset/09_c
Moved files from dataset/00\10_down to reorganized_dataset/10_down
Moved files from dataset/01\01_palm to reorganized_dataset/01_palm
Moved files from dataset/01\02_l to reorganized_dataset/02_l
Moved files from dataset/01\03_fist to reorganized_dataset/03_fist
Moved files from dataset/01\04_fist_moved to reorganized_dataset/04_fist_moved
Moved files from dataset/01\05_thumb to reor

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32
epochs = 10

In [3]:
train_data_dir = 'reorganized_dataset//' 

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [5]:
from tensorflow.keras.applications import MobileNet

In [8]:
mobilenet = MobileNet(weights='imagenet',include_top=False,input_shape=input_shape)

In [9]:
for layer in mobilenet.layers:
    layer.trainable = False

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [11]:
model = Sequential([
    mobilenet,
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [12]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',   # Metric to monitor
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored metric
    verbose=1,            # Verbosity mode
    mode='min'            # Mode can be 'min', 'max', or 'auto'
)

In [11]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        epochs=epochs,
        callbacks=[early_stopping]
    )

Epoch 1/10
500/500 [==============================] - 665s 1s/step - loss: 0.1034 - accuracy: 0.9655 - val_loss: 0.3742 - val_accuracy: 0.9277
Epoch 2/10
500/500 [==============================] - 525s 1s/step - loss: 0.0131 - accuracy: 0.9961 - val_loss: 0.0863 - val_accuracy: 0.9775
Epoch 3/10
500/500 [==============================] - 382s 765ms/step - loss: 0.0155 - accuracy: 0.9962 - val_loss: 0.9458 - val_accuracy: 0.8570
Epoch 4/10
500/500 [==============================] - 466s 933ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.1200 - val_accuracy: 0.9707
Epoch 5/10
500/500 [==============================] - 374s 749ms/step - loss: 6.9957e-04 - accuracy: 0.9996 - val_loss: 0.2614 - val_accuracy: 0.9563
Epoch 6/10
500/500 [==============================] - 374s 749ms/step - loss: 0.0382 - accuracy: 0.9934 - val_loss: 0.4629 - val_accuracy: 0.9395
Epoch 7/10
500/500 [==============================] - 373s 747ms/step - loss: 0.0050 - accuracy: 0.9988 - val_loss: 0.4511 - v

In [ ]:
model.save('hand_recognition.h5')

In [14]:
from tensorflow.keras.models import load_model

In [15]:
model = load_model('hand_recognition.h5')

In [16]:
import cv2

In [25]:
cap = cv2.VideoCapture(0)

In [28]:
import numpy as np

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame is captured
    if not ret:
        break

    # Preprocess the frame
    frame_resized = cv2.resize(frame, (224, 224))  # Resize to 224x224
    frame_normalized = frame_resized / 255.0  # Normalize pixel values
    frame_reshaped = frame_normalized.reshape(1, 224, 224, 3)  # Add batch dimension

    # Predict the gesture
    predictions = model.predict(frame_reshaped)
    predicted_class = np.argmax(predictions, axis=1)

    gesture_labels = ['palm', 'I', 'fist', 'fist_moved', 'thumb', 'index', 'ok', 'palm moved', 'c', 'down'] 
    predicted_gesture = gesture_labels[predicted_class[0]]

    # Display the result
    cv2.putText(frame, f'Gesture: {predicted_gesture}', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Real-Time Gesture Recognition', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 0s 42ms/step


In [29]:
cap.release()
cv2.destroyAllWindows()